In [1]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [2]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html

In [3]:
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)

In [4]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [5]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [6]:
df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 무시무시 마녀가 이사 와!
1,http://www.hanbit.co.kr/store/books/look.php?p...,"받침 없는 한글 동화 : 도, 도, 도깨비다!"
2,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 바쁘다 바빠 너구리 바빠
3,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 도깨비 파자마 파티
4,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 4권 세트
5,http://www.hanbit.co.kr/store/books/look.php?p...,오늘도 시작하지 못하는 당신을 위해
6,http://www.hanbit.co.kr/store/books/look.php?p...,리얼 몽골 [2022~2023 최신판]
7,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, 원리로 이해하는 전력전자공학(2판)"
8,http://www.hanbit.co.kr/store/books/look.php?p...,트러스트
9,http://www.hanbit.co.kr/store/books/look.php?p...,리얼 싱가포르 [2022~2023 최신판]


# scrape_by_lxml

In [7]:
!pip install lxml
!pip install cssselect

In [8]:
import lxml.html

In [9]:
# HTML 파일을 읽어 들이고, getroot() 메서드로 HtmlElement 객체 생성
tree = lxml.html.parse('dp.html')
html = tree.getroot()

In [10]:
# cssselect() 메서드로 a 요소의 리스트를 추출 및 반복 수행
for a in html.cssselect('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)

#gnb None
#top_search None
#container None
https://www.hanbit.co.kr/index.html None
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
https://www.hanbit.co.kr/store/store_submain.html STORE
https://www.hanbit.co.kr/support/help_info.html SUPPORT
https://www.hanbit.co.kr/event/current/current_event_list.

# scrape_by_bs4

In [12]:
from bs4 import BeautifulSoup

In [16]:
# HTML 파일을 읽어 들이고 BeautifulSoup 객체를 생성
with open('dp.html', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [17]:
# find_all() 메서드로 a 요소를 추출 및 반복 수행
for a in soup.find_all('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)

#gnb 메뉴 바로가기
#top_search 검색 및 카테고리 바로가기
#container 본문 바로가기
https://www.hanbit.co.kr/index.html HOME
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
https://www.hanbit.co.kr/store/store_submain.html STORE
https://www.hanbit.co.kr/support/help_info.html SUPPORT
https://www.hanbit.co.kr/event/current/cur